## Import & install


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install statsforecast s3fs fastparquet
!pip install git+https://github.com/Nixtla/neuralforecast.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.0/275.0 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.2/169.2 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.6/154.6 kB 21.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.5/298.5 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 18.3 MB/s eta 0

In [3]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import os
import random
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from ray import tune

from neuralforecast import NeuralForecast
from neuralforecast.auto import AutoNHITS, AutoTFT
from neuralforecast.losses.pytorch import DistributionLoss
import pandas as pd
from statsforecast import StatsForecast as sf

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42) # Seed 고정

# 전처리

In [6]:
train_data = pd.read_csv('/content/drive/MyDrive/LG_AIMERS/train.csv').drop(columns=['ID', '제품'])
keyword = pd.read_csv('/content/drive/MyDrive/LG_AIMERS/brand_keyword_cnt.csv')
product = pd.read_csv('/content/drive/MyDrive/LG_AIMERS/product_info.csv')
sales = pd.read_csv('/content/drive/MyDrive/LG_AIMERS/sales.csv')
submission = pd.read_csv('/content/drive/MyDrive/LG_AIMERS/sample_submission.csv')


## 최근 30일이 0인것 -> sub1

In [7]:
to_zero_index = train_data[train_data.iloc[:,-21:].mean(axis=1)==0].index

In [8]:
submission1 = submission.loc[to_zero_index]

## 마지막 7일 중 6일이 0 이고, 마지막 값만 존재하는경우 -> sub2

In [9]:
mean_30days_over_zero = list(set(submission.index) - set(submission1.index))

data1 = train_data.loc[mean_30days_over_zero]

condition = (data1.iloc[:, -7:-1].sum(axis=1) == 0) & (data1.iloc[:, -1] > 0)
filtered_data = data1[condition]

only_one_value_exist_index = filtered_data.index

real_value_last_one = train_data.loc[only_one_value_exist_index].iloc[:,-1]

In [10]:
submission2 = submission.loc[only_one_value_exist_index]
submission2.iloc[:,1:] = submission2.iloc[:,1:].apply(lambda _: real_value_last_one, axis=0)

# 최종 데이터

In [11]:
last_index = set(submission.index) - set(submission1.index) - set(submission2.index)

In [12]:
len(last_index)

5430

In [13]:
data_2023 = train_data.loc[last_index]
ID_2023 = data_2023.index
data_2023.reset_index(drop=True,inplace=True)

In [14]:
ID_2023

Int64Index([    1,    17,    19,    29,    30,    32,    45,    47,    48,
               50,
            ...
            15873, 15875, 15876, 15877, 15879, 15880, 15881, 15882, 15883,
            15886],
           dtype='int64', length=5430)

In [15]:
data_2023

,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,1,3,2,0,0,2,0
1,B002-C001-0002,B002-C002-0006,B002-C003-0032,B002-00005,0,0,0,0,0,0,...,0,0,0,0,25,0,10,0,0,0
2,B002-C001-0002,B002-C002-0006,B002-C003-0032,B002-00005,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,B002-C001-0002,B002-C002-0005,B002-C003-0025,B002-00007,0,0,0,0,0,0,...,0,0,0,4,5,3,2,1,4,0
4,B002-C001-0001,B002-C002-0001,B002-C003-0001,B002-00008,0,0,3,4,4,3,...,0,0,0,0,0,1,2,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5425,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03798,0,0,0,0,0,0,...,0,0,0,2,0,8,0,0,0,5
5426,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03798,0,0,0,0,0,0,...,2,2,1,0,0,0,0,0,5,1
5427,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03798,0,0,0,0,0,0,...,3,2,2,0,0,0,0,0,1,0
5428,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03798,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,2,0


# 앞에서부터 0인 데이터를 제거하고, 이상치 제거와 rolling 을 통해서 데이터를 부드럽게 해주는 과정

In [16]:
train_data_info = data_2023.iloc[:,:4]
train_data_values = data_2023.iloc[:,4:]
values = train_data_values.T

In [17]:
def hampel_filter(data, window_size, num_std_dev):
    median = data.rolling(window=window_size, center=True).median()
    deviation = np.abs(data - median)
    median_deviation = deviation.rolling(window=window_size, center=True).median()
    threshold = num_std_dev * 1.4826 * median_deviation
    outlier_mask = deviation > threshold

    filtered_data = data.copy()
    filtered_data[outlier_mask] = median[outlier_mask]

    return filtered_data

In [18]:
window_size = 7
num_std_dev = 3
filtered_values = hampel_filter(values, window_size, num_std_dev)

In [19]:
filtered_values

,0,1,2,3,4,5,6,7,8,9,...,5420,5421,5422,5423,5424,5425,5426,5427,5428,5429
2022-01-01,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-02,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-03,0,0,0,0,3,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-04,0,0,0,0,4,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-05,0,0,0,0,4,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-31,2,0,0,3,1,13,1,5,0,9,...,0,0,5,0,0,8,0,0,0,2
2023-04-01,0,10,0,2,2,0,3,0,0,7,...,0,0,0,0,0,0,0,0,0,4
2023-04-02,0,0,0,1,2,9,5,17,0,9,...,0,0,2,0,5,0,0,0,0,1
2023-04-03,2,0,0,4,2,12,1,7,1,1,...,1,1,4,0,4,0,5,1,2,1


In [20]:
df = filtered_values.T
df = df.T
df = df.rolling(window=7,min_periods=1).mean()

In [21]:
data = df

In [22]:
a = data.T

In [23]:
ID_2023

Int64Index([    1,    17,    19,    29,    30,    32,    45,    47,    48,
               50,
            ...
            15873, 15875, 15876, 15877, 15879, 15880, 15881, 15882, 15883,
            15886],
           dtype='int64', length=5430)

In [24]:
pd.DataFrame(ID_2023)

,0
0,1
1,17
2,19
3,29
4,30
...,...
5425,15880
5426,15881
5427,15882
5428,15883


In [25]:
data1 = pd.concat([pd.DataFrame(ID_2023),a],axis=1)

In [26]:
data1 = data1.rename(columns ={0 : 'ID'})

In [27]:
high_data = data1[data1.iloc[:,1:].mean(axis=1)>30]

In [28]:
low_data =data1.loc[set(data1.index)-set(high_data.index)]

In [29]:
low_data

,ID,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,1,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.142857,0.571429,0.857143,0.857143,0.857143,1.142857,1.142857
1,17,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,3.571429,3.571429,5.000000,5.000000,5.000000,5.000000
2,19,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
3,29,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,1.142857,...,0.000000,0.000000,0.000000,0.285714,1.000000,1.428571,1.714286,1.857143,2.428571,2.428571
4,30,0.0,0.0,1.0,1.75,2.2,2.333333,2.0,2.142857,2.285714,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.428571,0.714286,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5425,15880,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.142857,1.142857,1.142857,1.142857,1.857143
5426,15881,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,...,1.714286,1.714286,1.714286,1.428571,1.000000,0.857143,0.571429,0.285714,0.857143,0.857143
5427,15882,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,...,2.857143,2.857143,2.714286,2.285714,1.857143,1.428571,1.000000,0.571429,0.428571,0.142857
5428,15883,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,...,3.428571,3.000000,2.571429,1.857143,1.285714,0.714286,0.285714,0.000000,0.285714,0.285714


In [30]:
high_data

,ID,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
57,207,77.0,77.0,73.000000,68.25,62.0,59.000000,58.142857,54.714286,52.714286,...,0.000000,0.000000,0.000000,1.000000,3.714286,4.714286,9.142857,9.285714,10.428571,12.000000
58,208,53.0,42.5,39.000000,36.50,37.0,36.333333,36.714286,36.857143,42.428571,...,0.000000,0.000000,0.000000,2.571429,5.571429,8.428571,10.428571,13.000000,14.428571,17.857143
67,224,4.0,3.5,2.666667,2.75,2.8,3.166667,3.142857,3.000000,3.000000,...,16.857143,16.285714,15.285714,15.857143,15.142857,17.571429,17.142857,17.571429,24.285714,29.714286
75,248,0.0,2.0,4.000000,5.00,7.2,8.833333,9.285714,10.428571,10.428571,...,0.000000,0.000000,0.000000,0.571429,5.142857,9.714286,13.428571,18.571429,22.285714,28.857143
77,251,36.0,30.0,24.000000,57.00,86.4,106.000000,99.428571,97.714286,97.714286,...,0.000000,0.000000,0.000000,7.285714,14.571429,32.285714,42.571429,56.857143,64.857143,80.857143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5363,15728,59.0,55.5,49.000000,51.50,53.8,56.000000,56.857143,56.857143,57.285714,...,0.000000,0.000000,0.000000,3.285714,6.714286,8.714286,11.142857,14.000000,16.857143,19.571429
5377,15788,0.0,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,9.857143,25.285714,58.000000,67.857143,107.714286,141.571429,187.428571
5384,15807,0.0,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000,3.000000,4.000000
5389,15821,105.0,115.0,104.000000,98.00,82.8,72.833333,68.000000,58.571429,46.142857,...,0.000000,0.000000,0.000000,3.571429,12.857143,16.428571,22.571429,35.714286,40.285714,51.571429


# high_data

In [31]:
final_data1 = high_data

In [32]:
high_ID_list = final_data1['ID']

In [33]:
final_data1 = final_data1.set_index('ID')

In [34]:
final_data1

,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
ID,,,,,,,,,,,,,,,,,,,,,
207,77.0,77.0,73.000000,68.25,62.0,59.000000,58.142857,54.714286,52.714286,52.571429,...,0.000000,0.000000,0.000000,1.000000,3.714286,4.714286,9.142857,9.285714,10.428571,12.000000
208,53.0,42.5,39.000000,36.50,37.0,36.333333,36.714286,36.857143,42.428571,48.000000,...,0.000000,0.000000,0.000000,2.571429,5.571429,8.428571,10.428571,13.000000,14.428571,17.857143
224,4.0,3.5,2.666667,2.75,2.8,3.166667,3.142857,3.000000,3.000000,3.428571,...,16.857143,16.285714,15.285714,15.857143,15.142857,17.571429,17.142857,17.571429,24.285714,29.714286
248,0.0,2.0,4.000000,5.00,7.2,8.833333,9.285714,10.428571,10.428571,9.857143,...,0.000000,0.000000,0.000000,0.571429,5.142857,9.714286,13.428571,18.571429,22.285714,28.857143
251,36.0,30.0,24.000000,57.00,86.4,106.000000,99.428571,97.714286,97.714286,99.428571,...,0.000000,0.000000,0.000000,7.285714,14.571429,32.285714,42.571429,56.857143,64.857143,80.857143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15728,59.0,55.5,49.000000,51.50,53.8,56.000000,56.857143,56.857143,57.285714,60.285714,...,0.000000,0.000000,0.000000,3.285714,6.714286,8.714286,11.142857,14.000000,16.857143,19.571429
15788,0.0,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,9.857143,25.285714,58.000000,67.857143,107.714286,141.571429,187.428571
15807,0.0,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000,3.000000,4.000000


In [35]:
data1 = final_data1.T

In [36]:
df = data1

# 머신러닝 데이터셋으로 변환
ml_dataset = []
for date, row in df.iterrows():
    for column, value in row.items():
        ml_dataset.append({'date': date, 'ID': column, 'values': value})

ml_df = pd.DataFrame(ml_dataset)

In [37]:
ml_df.columns  = ['ds','unique_id','y']

In [38]:
ml_df

,ds,unique_id,y
0,2022-01-01,207,77.000000
1,2022-01-01,208,53.000000
2,2022-01-01,224,4.000000
3,2022-01-01,248,0.000000
4,2022-01-01,251,36.000000
...,...,...,...
383260,2023-04-04,15728,19.571429
383261,2023-04-04,15788,187.428571
383262,2023-04-04,15807,4.000000
383263,2023-04-04,15821,51.571429


In [39]:
from neuralforecast.losses.pytorch import SMAPE
from neuralforecast.losses.pytorch import MAE


In [40]:
def to_dataset(df):
  ml_dataset = []
  for date, row in df.iterrows():
      for column, value in row.items():
          ml_dataset.append({'date': date, 'ID': column, 'values': value})

  ml_df = pd.DataFrame(ml_dataset)
  ml_df.columns  = ['ds','unique_id','y']
  return ml_df

config_nhits = {
    "input_size": tune.choice([21, 21*2, 21*3,21*4,21*5]),              # Length of input window
    "n_blocks": 5*[1],                                              # Length of input window
    "mlp_units": 5 * [[512, 512]],                                  # Length of input window
    "n_pool_kernel_size": tune.choice([4*[1], 4*[2], 4*[4],
                                       8*[1], 8*[2], 8*[4],
                                        [16, 8, 1],
                                       16*[1], 16*[2], 16*[4],
                                      [8, 4, 2, 1, 1]]),            # MaxPooling Kernel size
    "n_freq_downsample": tune.choice([[8, 4, 2, 1, 1],
                                      [1, 1, 1, 1, 1]]),            # Interpolation expressivity ratios
    "learning_rate": tune.loguniform(1e-4, 1e-2),                   # Initial Learning rate
    "scaler_type": None, #tune.choice([MinMax]),                             # Scaler type
    "max_steps": tune.choice([500]),                               # Max number of training iterations
    "batch_size": tune.choice([32,64,256,512,1024]),                  # Number of series in batch
    "windows_batch_size": tune.choice([32,64,96,128,256,512]),       # Number of windows in batch
    "random_seed": 42,                             # Random seed
    "val_check_steps": 50,                                                    # Compute validation every 50 steps
    }

In [41]:
torch.cuda.is_available()


True

In [42]:
ID_LIST = ml_df['unique_id'].unique()

In [43]:
ml_df['ds'] = pd.to_datetime(ml_df['ds'])

In [44]:
ml_df

,ds,unique_id,y
0,2022-01-01,207,77.000000
1,2022-01-01,208,53.000000
2,2022-01-01,224,4.000000
3,2022-01-01,248,0.000000
4,2022-01-01,251,36.000000
...,...,...,...
383260,2023-04-04,15728,19.571429
383261,2023-04-04,15788,187.428571
383262,2023-04-04,15807,4.000000
383263,2023-04-04,15821,51.571429


In [45]:
model = AutoNHITS(h=21,
                  loss = SMAPE(),
                  config=config_nhits,
                  #search_alg=HyperOptSearch(),
                  num_samples=100)

In [46]:
nf = NeuralForecast(models=[model], freq='D')
nf.fit(df=ml_df, val_size=21)

2023-09-01 12:09:05,730	INFO worker.py:1621 -- Started a local Ray instance.
2023-09-01 12:09:07,387	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
2023-09-01 12:09:07,390	INFO tune.py:666 -- [output] This will use the new output engine with verbosity 0. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


+--------------------------------------------------------------------+
| Configuration for experiment     _train_tune_2023-09-01_12-09-03   |
+--------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator             |
| Scheduler                        FIFOScheduler                     |
| Number of trials                 100                               |
+--------------------------------------------------------------------+

View detailed results here: /root/ray_results/_train_tune_2023-09-01_12-09-03
To visualize your results with TensorBoard, run: `tensorboard --logdir /root/ray_results/_train_tune_2023-09-01_12-09-03`



(_train_tune pid=2485) Global seed set to 42
(_train_tune pid=2485) 2023-09-01 12:09:12.885899: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
(_train_tune pid=2485) To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
(_train_tune pid=2485) 2023-09-01 12:09:13.896440: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Sanity Checking: 0it [00:00, ?it/s]
Epoch 3:   0%|          | 0/14 [00:00<?, ?it/s, v_num=0, train_loss_step=0.835, train_loss_epoch=0.522]
(_train_tune pid=2485) 
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|          | 0/14 [00:00<?, ?it/s]
(_train_tune pid=2485) 
Epoch 3:   0%|          | 0/14 [00:00<?, ?it/s, v_num=0, train_loss_step=0.684, train_loss_epoch=0.522, valid_loss=1.580]
                                                                         
Epoch 7:   0%|          | 0/14 [00:00<?, ?it/s, v_num=0, train_loss_step=0.477, train_loss_epoch=0.529, valid_loss=1.580]
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|          | 0/14 [00:00<?, ?it/s]
(_train_tune pid=2485) 
Epoch 7:   0%|          | 0/14 [00:00<?, ?it/s, v_num=0, train_loss_step=0.456, train_loss_epoch=0.529, valid_loss=1.580]
                                                                         
Epoch 10:   0%|          | 0/14 [00:00<?, ?it/s, v_num=0, train_loss_step=0.574, tra

(_train_tune pid=2485) Global seed set to 42


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [47]:
fcst_df = nf.predict()


Predicting: 0it [00:00, ?it/s]

In [48]:
fcst_df

,ds,AutoNHITS
unique_id,,
207,2023-04-05,13.963117
207,2023-04-06,13.781285
207,2023-04-07,13.681452
207,2023-04-08,14.138242
207,2023-04-09,14.781242
...,...,...
15823,2023-04-21,48.494934
15823,2023-04-22,48.971188
15823,2023-04-23,49.357216


In [49]:
config1 = nf.models[0].results.get_best_result().config

In [50]:
config1

{'input_size': 42,
 'n_blocks': [1, 1, 1, 1, 1],
 'mlp_units': [[512, 512], [512, 512], [512, 512], [512, 512], [512, 512]],
 'n_pool_kernel_size': [4, 4, 4, 4, 4, 4, 4, 4],
 'n_freq_downsample': [8, 4, 2, 1, 1],
 'learning_rate': 0.0005252684100000482,
 'scaler_type': None,
 'max_steps': 500,
 'batch_size': 512,
 'windows_batch_size': 256,
 'random_seed': 42,
 'val_check_steps': 50,
 'h': 21,
 'loss': SMAPE(),
 'valid_loss': SMAPE()}

In [51]:
final_data = fcst_df.reset_index()
ids = final_data['unique_id'].unique()

In [52]:
p = pd.DataFrame()
for i in ids:
  k = final_data[final_data['unique_id']==i]['AutoNHITS']
  k = pd.DataFrame(k).T
  k.columns = submission.iloc[:,1:].columns
  p = pd.concat([p,k],axis=0)

In [53]:
p = p.reset_index(drop=True)

In [54]:
high_ID_list

57        207
58        208
67        224
75        248
77        251
        ...  
5363    15728
5377    15788
5384    15807
5389    15821
5391    15823
Name: ID, Length: 835, dtype: int64

In [55]:
p.index = high_ID_list

In [56]:
submission5 = p.reset_index()

# low_data

In [57]:
final_data1 = low_data

In [58]:
high_ID_list = final_data1['ID']

In [59]:
final_data1 = final_data1.set_index('ID')

In [60]:
final_data1

,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
ID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.142857,0.571429,0.857143,0.857143,0.857143,1.142857,1.142857
17,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,3.571429,3.571429,5.000000,5.000000,5.000000,5.000000
19,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
29,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,1.142857,2.0,...,0.000000,0.000000,0.000000,0.285714,1.000000,1.428571,1.714286,1.857143,2.428571,2.428571
30,0.0,0.0,1.0,1.75,2.2,2.333333,2.0,2.142857,2.285714,2.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.428571,0.714286,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15880,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.142857,1.142857,1.142857,1.142857,1.857143
15881,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,1.714286,1.714286,1.714286,1.428571,1.000000,0.857143,0.571429,0.285714,0.857143,0.857143
15882,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,2.857143,2.857143,2.714286,2.285714,1.857143,1.428571,1.000000,0.571429,0.428571,0.142857


In [61]:
data1 = final_data1.T

In [62]:
df = data1

# 머신러닝 데이터셋으로 변환
ml_dataset = []
for date, row in df.iterrows():
    for column, value in row.items():
        ml_dataset.append({'date': date, 'ID': column, 'values': value})

ml_df = pd.DataFrame(ml_dataset)

In [63]:
ml_df.columns  = ['ds','unique_id','y']

In [64]:
ml_df

,ds,unique_id,y
0,2022-01-01,1,0.000000
1,2022-01-01,17,0.000000
2,2022-01-01,19,0.000000
3,2022-01-01,29,0.000000
4,2022-01-01,30,0.000000
...,...,...,...
2109100,2023-04-04,15880,1.857143
2109101,2023-04-04,15881,0.857143
2109102,2023-04-04,15882,0.142857
2109103,2023-04-04,15883,0.285714


In [65]:
from neuralforecast.losses.pytorch import SMAPE
from neuralforecast.losses.pytorch import MAE


In [66]:
def to_dataset(df):
  ml_dataset = []
  for date, row in df.iterrows():
      for column, value in row.items():
          ml_dataset.append({'date': date, 'ID': column, 'values': value})

  ml_df = pd.DataFrame(ml_dataset)
  ml_df.columns  = ['ds','unique_id','y']
  return ml_df

config_nhits = {
    "input_size": tune.choice([21, 21*2, 21*3,21*4,21*5]),              # Length of input window
    "n_blocks": 5*[1],                                              # Length of input window
    "mlp_units": 5 * [[512, 512]],                                  # Length of input window
    "n_pool_kernel_size": tune.choice([4*[1], 4*[2], 4*[4],
                                       8*[1], 8*[2], 8*[4],
                                        [16, 8, 1],
                                       16*[1], 16*[2], 16*[4],
                                      [8, 4, 2, 1, 1]]),            # MaxPooling Kernel size
    "n_freq_downsample": tune.choice([[8, 4, 2, 1, 1],
                                      [1, 1, 1, 1, 1]]),            # Interpolation expressivity ratios
    "learning_rate": tune.loguniform(1e-4, 1e-2),                   # Initial Learning rate
    "scaler_type": None, #tune.choice([MinMax]),                             # Scaler type
    "max_steps": tune.choice([500]),                               # Max number of training iterations
    "batch_size": tune.choice([32,64,256,512,1024]),                  # Number of series in batch
    "windows_batch_size": tune.choice([32,64,96,128,256,512]),       # Number of windows in batch
    "random_seed": 42,                             # Random seed
    "val_check_steps": 50,                                                    # Compute validation every 50 steps
    }

In [67]:
torch.cuda.is_available()


True

In [68]:
ID_LIST = ml_df['unique_id'].unique()

In [69]:
ml_df['ds'] = pd.to_datetime(ml_df['ds'])

In [70]:
ml_df

,ds,unique_id,y
0,2022-01-01,1,0.000000
1,2022-01-01,17,0.000000
2,2022-01-01,19,0.000000
3,2022-01-01,29,0.000000
4,2022-01-01,30,0.000000
...,...,...,...
2109100,2023-04-04,15880,1.857143
2109101,2023-04-04,15881,0.857143
2109102,2023-04-04,15882,0.142857
2109103,2023-04-04,15883,0.285714


In [71]:
model = AutoNHITS(h=21,
                  loss = SMAPE(),
                  config=config_nhits,
                  #search_alg=HyperOptSearch(),
                  num_samples=100)

In [72]:
nf = NeuralForecast(models=[model], freq='D')
nf.fit(df=ml_df, val_size=21)

2023-09-01 12:28:26,917	INFO tune.py:666 -- [output] This will use the new output engine with verbosity 0. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


+--------------------------------------------------------------------+
| Configuration for experiment     _train_tune_2023-09-01_12-28-26   |
+--------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator             |
| Scheduler                        FIFOScheduler                     |
| Number of trials                 100                               |
+--------------------------------------------------------------------+

View detailed results here: /root/ray_results/_train_tune_2023-09-01_12-28-26
To visualize your results with TensorBoard, run: `tensorboard --logdir /root/ray_results/_train_tune_2023-09-01_12-28-26`



(_train_tune pid=7810) Global seed set to 42
(_train_tune pid=7810) 2023-09-01 12:28:32.452426: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
(_train_tune pid=7810) To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
(_train_tune pid=7810) 2023-09-01 12:28:33.564727: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Epoch 0:  28%|██▊       | 40/144 [00:00<00:01, 71.34it/s, v_num=0, train_loss_step=1.040]
(_train_tune pid=7810) 
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|          | 0/144 [00:00<?, ?it/s]
(_train_tune pid=7810) 
Validation DataLoader 0:  14%|█▍        | 20/144 [00:00<00:00, 145.86it/s]
(_train_tune pid=7810) 
Validation DataLoader 0:  28%|██▊       | 40/144 [00:00<00:00, 145.35it/s]
(_train_tune pid=7810) 
Validation DataLoader 0:  42%|████▏     | 60/144 [00:00<00:00, 143.67it/s]
(_train_tune pid=7810) 
Validation DataLoader 0:  56%|█████▌    | 80/144 [00:00<00:00, 144.10it/s]
(_train_tune pid=7810) 
Validation DataLoader 0:  69%|██████▉   | 100/144 [00:00<00:00, 142.99it/s]
(_train_tune pid=7810) 
Validation DataLoader 0:  83%|████████▎ | 120/144 [00:00<00:00, 143.00it/s]
(_train_tune pid=7810) 
Epoch 0:  28%|██▊       | 40/144 [00:01<00:04, 23.60it/s, v_num=0, train_loss_step=1.030, valid_loss=1.570]
                                                              

(_train_tune pid=7810) Global seed set to 42


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [73]:
fcst_df = nf.predict()


Predicting: 0it [00:00, ?it/s]

In [74]:
config2 = nf.models[0].results.get_best_result().config

In [75]:
config2

{'input_size': 84,
 'n_blocks': [1, 1, 1, 1, 1],
 'mlp_units': [[512, 512], [512, 512], [512, 512], [512, 512], [512, 512]],
 'n_pool_kernel_size': [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
 'n_freq_downsample': [1, 1, 1, 1, 1],
 'learning_rate': 0.0017392589033005073,
 'scaler_type': None,
 'max_steps': 500,
 'batch_size': 1024,
 'windows_batch_size': 96,
 'random_seed': 42,
 'val_check_steps': 50,
 'h': 21,
 'loss': SMAPE(),
 'valid_loss': SMAPE()}

In [76]:
final_data = fcst_df.reset_index()
ids = final_data['unique_id'].unique()

In [77]:
p = pd.DataFrame()
for i in ids:
  k = final_data[final_data['unique_id']==i]['AutoNHITS']
  k = pd.DataFrame(k).T
  k.columns = submission.iloc[:,1:].columns
  p = pd.concat([p,k],axis=0)

In [78]:
p = p.reset_index(drop=True)

In [79]:
high_ID_list

0           1
1          17
2          19
3          29
4          30
        ...  
5425    15880
5426    15881
5427    15882
5428    15883
5429    15886
Name: ID, Length: 4595, dtype: int64

In [80]:
p.index = high_ID_list

In [81]:
submission6 = p.reset_index()

In [82]:
final_data = pd.concat([submission1,submission2,submission5,submission6],axis=0)

In [83]:
final_data.iloc[:,1:] = final_data.iloc[:,1:].replace(0,1)

In [84]:
final_data

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,4,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
5,5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4590,15880,1.849985,1.807898,1.777036,1.793206,1.816262,1.859486,1.880378,1.922233,1.876742,...,1.743173,1.807531,1.778327,1.795138,1.807171,1.843981,1.901219,1.922512,1.897756,1.902144
4591,15881,0.908154,0.938016,0.982216,1.029023,1.086766,1.184267,1.233642,1.284530,1.257916,...,1.205065,1.268680,1.288828,1.316824,1.327176,1.351304,1.395617,1.411010,1.386613,1.373688
4592,15882,0.295674,0.427495,0.532311,0.670668,0.755976,0.963515,1.063177,1.172935,1.153610,...,1.219017,1.333081,1.382106,1.390077,1.401278,1.432219,1.532609,1.544771,1.553424,1.544233
4593,15883,0.474047,0.607090,0.745614,0.872269,0.968275,1.198593,1.299909,1.424639,1.421204,...,1.492090,1.600430,1.661317,1.683751,1.687740,1.706090,1.817974,1.834941,1.841160,1.838276


In [85]:
#final_data.iloc[:,1:] = np.round(final_data.iloc[:,1:],0).astype(int)

In [86]:
final_submission = final_data.sort_values(by='ID')

In [87]:
final_submission.iloc[:,1:] = np.round(final_submission.iloc[:,1:],0).astype(int)

In [88]:
final_submission = final_submission.reset_index(drop=True)

In [89]:
final_submission.to_csv('/content/drive/MyDrive/LG_AIMERS/nhits_단일.csv', index=False)